Notes: 
* Answer each question in a separate Jupynet Notebook Cell
* Pleas keep the code in your cells short. 
  * In notebook programming cells are typicaly short to facilitate reading. 
  * If well toughout, most answers in this assignment won're require more than 3 or 4 lines of code. 
* Do no change the list of import, i.e., do not add additional libraries. Those included are the only ones you are allowed to use.
* Add your first and Last name below:

Jun Miao
    


In [1]:
import pandas as pd
from itertools import product
from collections import Counter
from tqdm.notebook import tqdm
import random

In this assignment you will be working with Corona Virus (SARS-CoV2) data that was obtained from the [National Center for Biotechnology Information](https://www.ncbi.nlm.nih.gov/). You will need two files. The first (`data/coronavirus_info.csv`) is small and is provided in the GitHub Repo. The second  (`data_report.jsonl`) is larger so you will need to download a compressed version, which you will need to uncompress prior to using. You can downlod the second file here:

https://www.dropbox.com/s/qdn67rshygz06ff/data_report.jsonl.gz?dl=0

We start by loading `data/coronavirus_info.csv` (Code provided below)

In [2]:
# CODE PROVIDED -- DO NOT REMOVE
table = pd.read_csv("data/coronavirus_info.csv", low_memory=False)
table = table.drop(["US State", "Host Name", "Host Taxonomy ID", "Sequence Type", "Species Taxonomy Id", "Nuc Completeness", "BioProject", "BioSample"], axis=1)

missing = table["Geo Location"].isnull()
table.loc[missing, "Geo Location"] = ""


table.head(10)

,Nucleotide Accession,Species Name,Virus Genus,Virus Family,Isolate Name,Nucleotide Length,Geo Location,Collection Date
0,NC_045512.2,Severe acute respiratory syndrome coronavirus 2,Betacoronavirus,Coronaviridae,Wuhan-Hu-1,29903,Asia; China,2019-12
1,OK058807.1,Severe acute respiratory syndrome coronavirus 2,Betacoronavirus,Coronaviridae,SARS-CoV-2/human/USA/MA-MASPHL-04825/2021,29801,North America; USA,2021-07-29
2,OK058777.1,Severe acute respiratory syndrome coronavirus 2,Betacoronavirus,Coronaviridae,SARS-CoV-2/human/USA/MA-MASPHL-04790/2021,29771,North America; USA,2021-08-10
3,OK058695.1,Severe acute respiratory syndrome coronavirus 2,Betacoronavirus,Coronaviridae,SARS-CoV-2/human/USA/MA-MASPHL-04700/2021,29820,North America; USA,2021-08-15
4,OK058662.1,Severe acute respiratory syndrome coronavirus 2,Betacoronavirus,Coronaviridae,SARS-CoV-2/human/USA/MA-MASPHL-04651/2021,29798,North America; USA,2021-08-09
5,OK058592.1,Severe acute respiratory syndrome coronavirus 2,Betacoronavirus,Coronaviridae,SARS-CoV-2/human/USA/MA-MASPHL-04499/2021,29802,North America; USA,2021-07-26
6,OK056996.1,Severe acute respiratory syndrome coronavirus 2,Betacoronavirus,Coronaviridae,SARS-CoV-2/human/USA/FL-CDC-QDX27934346/2021,29775,North America; USA: Florida,2021-08-18
7,OK056909.1,Severe acute respiratory syndrome coronavirus 2,Betacoronavirus,Coronaviridae,SARS-CoV-2/human/USA/CA-CDC-QDX27909662/2021,29775,North America; USA: California,2021-08-16
8,OK056850.1,Severe acute respiratory syndrome coronavirus 2,Betacoronavirus,Coronaviridae,SARS-CoV-2/human/USA/FL-CDC-QDX27934406/2021,29763,North America; USA: Florida,2021-08-18
9,OK056784.1,Severe acute respiratory syndrome coronavirus 2,Betacoronavirus,Coronaviridae,SARS-CoV-2/human/USA/NY-CDC-QDX28007789/2021,29775,North America; USA: New York,2021-08-21


### Q.1

* The location of each of the sequences is recorded under the `Geo Location` column.  How many entries are from Asia?
  * Note that for some records, the `Geo Location` column is missing
  * Display the results using the following format: 
    Asia: XXXX,
    North America': XXXX,
    Europe: XXXX,
    Oceania: XXXX,
    Africa: XXXX,
    South America: XXXX 


In [3]:
result_dict = {'Asia': 0, 'North America': 0, 'Europe': 0, 'Oceania': 0, 'Africa': 0, 'South America': 0}

def region_count(i):
    for k in result_dict.keys():
        if k in i:
            result_dict[k] += 1

table['Geo Location'].apply(region_count)
print(f"Asia: {result_dict['Asia']}, North America: {result_dict['North America']}, Europe: {result_dict['Europe']}, "
      f"Oceania: {result_dict['Oceania']}, Africa: {result_dict['Africa']}, South America: {result_dict['South America']}")

Asia: 3903, North America: 224014, Europe: 189100, Oceania: 10301, Africa: 1405, South America: 534


### Q.2
Use the `coronavirus_info.csv` table to count the entries that are from Hawaii. 

In [4]:
def filter_hawaii(x):
    return 1 if 'Hawaii' in x else 0

hawaii_region = table['Geo Location'].apply(filter_hawaii)
hawaii_region.sum()

119

### Q.3

The file `data_report.jsonl` contains the variants of the virus in the DB. This `json` file is a list of records (one per line) for each one of the genomes in the database. Before we work with the large file, we will experiment with a file containing a single record.

The file `single_record.json` contains a single sample record. Use the `JSON library to load the file `single_record.json` into a variable called `sample_vir_record`

1. how many first-level keys does this record have?
  * Do not count nested keys. Only those at the top level should be counted



In [5]:
import json

sample_vir_record = json.load(open('./data/single_record.json', 'r'))
len(sample_vir_record.keys())

18

### Q.4

Each Covid various in this database is classified according to a system referred to as the Pangolin (Phylogenetic Assignment of Named Global Outbreak LINeages) classification. It is not essential to complete the assignment that you understand this system, but if you're interested in learning more, see:

https://cov-lineages.org/resources/pangolin.html

The Pangolin classification of this sample record is nested within the `virus` key:
```json
{ ...
  "virus": {
              ...
              "pangolinClassification": 
              ...
            }
  ... 
}
```
Write code to extract the classification of this record. The result should be `B.1.1.214`

In [6]:
sample_vir_record['virus']['pangolinClassification']

'B.1.1.214'

We hear in the news about `Alpha`, `Beta`, `Delta` variants of concern and recently the Mu variant as being of interest. Basedon your answer to `Q.3`, you may have been tempted to infer that this virus is of type `Beta` since the first letter is `B`. In fact, this variant of type `Alpha` and is a variant of concern. Although not relevant to this exercise, the rules for naming new variants and the list of known `SARS-CoV-2` are provided here:

https://www.pango.network/how-does-the-system-work/what-are-pango-lineages/

https://cov-lineages.org/lineage_list.html


The following short video is very helpful for understanding what a variant is, how it arises, how it's named, and why some variants are more concerning than others.

https://www.youtube.com/watch?v=B8UEZ9cfgz4

### Q.5

Write Python code that counts all the different kinds of variants in `data_report.jsonl`. 
Because the file is 7GB, you won't likely be able to load it into your laptop's RAM using Python. I encountered this error when trying to open it on my laptop

![](https://www.dropbox.com/s/lieo685pafkgm5e/ram_error.png?dl=1)


It would be easy to extract the data from the pangolinClassification field of each `json` record by reading each line (a record) at a time.

The list of current variants of concern we are interested in are:
      * Alpha (B.1.1.7)
      * Beta (B.1.351, B.1.351.2, B.1.351.3)
      * Delta (B.1.617.2, AY.1, AY.2, AY.3)
      * Gamma (P.1, P.1.1, P.1.2) 

You should get something similar to what follows:
```
Alpha: X
Beta: X
Delta: X
Gamma: X
```
Where `X` represents the counts for relevant variants

In [7]:
# Takes a while
data_report = {}

def variant_check(x):
    try:
        loaded = json.loads(x)
        return loaded['virus']['pangolinClassification']
    except KeyError:
        return

with open('./data/data_report.jsonl', 'r') as opened:
    for line in opened:
        temp = variant_check(line)
        if temp:
            try:
                data_report[temp] += 1
            except KeyError:
                data_report[temp] = 1

print(f"Alpha: {data_report['B.1.1.7']}")
print(f"Beta: {sum((data_report['B.1.351'], data_report['B.1.351.2'], data_report['B.1.351.3']))}")
print(f"Delta: {sum((data_report['B.1.617.2'], data_report['AY.1'], data_report['AY.2'], data_report['AY.3']))}")
print(f"Gamma: {sum((data_report['P.1'], data_report['P.1.1'], data_report['P.1.2']))}")

Alpha: 178423
Beta: 584
Delta: 4392
Gamma: 6632


# Find similar viruses.

It's often useful to compare viruses to study how similar strains are. While sophisticated algorithms to compare a pair of viruses exist, these are typically computationally intensive and cannot be used to carry out a large number of comparisons. 

An alternative, albeit less sensitive, approach consists of comparing word counts (called k-mers, where k is the word size) across genomes.  Suppose we have two viruses X and Y, with the following Genomes.
```
X = "ACGTAGTGCATGTGTAGCTGTGTAGCTGTAC"
Y = "ACTAGTGCATGTGTAGCTCTGTAGCTGATAC"
```

To compare `X` and `Y`, we first vectorize these genomes by marking the presence of words (k-mers) as a boolean value, 0 if absent and 1 if the word is present. This method assumes that similar genomes will have the same words, which makes sense.

This idea, which is referred to as the bag of words model is computationally efficient, making it ideal to vectorize text in big data analytics. Another variant of this model requires replacing the presence and absence by counts for each word.

The code below vectorizes an input DNA sequence intro k-mers of size k=2

In [8]:
# CODE PROVIDED -- DO NOT REMOVE
def get_kmer_2(X):
    DNA = ["A", "C", "G", "T"]
    words_size_2 = ["".join(dna_prod) for dna_prod in product(DNA, DNA)]
    counts = pd.Series([0 for _ in words_size_2], index = words_size_2)
    words_in_X = set([X[i:i+2] for i in range(0, len(X)-1)])
    counts[list(words_in_X)] = 1
    return counts    

In [9]:
# CODE PROVIDED -- DO NOT REMOVE
# X has 3 words of size 2 (AC, CG, GT)
X = "ACGT"
get_kmer_2("ACGT")

AA    0
AC    1
AG    0
AT    0
CA    0
CC    0
CG    1
CT    0
GA    0
GC    0
GG    0
GT    1
TA    0
TC    0
TG    0
TT    0
dtype: int64

The function below takes a dictionary of sequences' counts as a `pandas Series` and prints it using HTML Table, which you might agree is nicer to visualize.

In [10]:
# CODE PROVIDED -- DO NOT REMOVE
def pretty_print_counts(counts_dict):
    list_of_count = [data.to_list() for data in counts_dict.values()]
    list_of_indices = [x for x in counts_dict.keys()]
    list_of_columns = list(counts_dict.values())[0].index.to_list()
    df_single_level_cols = pd.DataFrame(list_of_count,
                                        index=[x for x in counts_dict.keys()],
                                       columns = list_of_columns)    
    return df_single_level_cols 



In [11]:
# CODE PROVIDED -- DO NOT REMOVE

X = "ACGTACGTACGTACGT"
Y = "ACGTACAAACGTACGT"
Z = "TTTTACAAACGTTTTT"

counts_dict = {"X": get_kmer_2(X), "Y": get_kmer_2(Y), "Z": get_kmer_2(Z)}
pretty_print_counts(counts_dict)


,AA,AC,AG,AT,CA,CC,CG,CT,GA,GC,GG,GT,TA,TC,TG,TT
X,0,1,0,0,0,0,1,0,0,0,0,1,1,0,0,0
Y,1,1,0,0,1,0,1,0,0,0,0,1,1,0,0,0
Z,1,1,0,0,1,0,1,0,0,0,0,1,1,0,0,1


### Q.6

Write a function that computes the Jaccard similarity between two feature vectors, A and B. If you recall, Jaccard similarity is computed as:

$$J(A,B) = \frac{A \cap B}{A \cup B}$$

In other words, the number of items shared by `A` and `B` over the set of all items in `A` or `B`.

For example, for `A= get_kmer_2(X)` and Y = get_kmer_2(B) above,

$$
J(A,B) = \frac{4}{6}
$$

Your function should have the following signature: 

`jaccard(A, B)`

Where `A` and `B` are `pandas Series`


Test your function using the code below to make sure it's correct.

In [12]:
def jaccard(A, B):
    top, bottom = 0, 0
    for i in A.keys():
        count = A[i] + B[i]
        if count > 0:
            bottom += 1
            if count > 1:
                top += 1
    return top / bottom

In [13]:
# TEST PROVIDED -- DO NOT REMOVE
A = get_kmer_2(X)
B = get_kmer_2(Y)
assert jaccard(A, B) == 4/6

### Q.7 

* Compute the jaccard similarity for the pairs of sequences `(X, Y)`, `(X, Z)`, `(Y, Z)`. 


In [14]:
A, B, C = get_kmer_2(X), get_kmer_2(Y), get_kmer_2(Z)
print(f'(X, Y) -> {jaccard(A, B)}\n(X, Z) -> {jaccard(A, C)}\n(Y, Z) -> {jaccard(B, C)}')

(X, Y) -> 0.6666666666666666
(X, Z) -> 0.5714285714285714
(Y, Z) -> 0.8571428571428571


### Q.8

The vectors representing the presence and absence of words in both `Y` and `Z` are very similar (Jaccard = 0.85), despite major differences at the DNA level between these two sequences. This is because the words are small -- it is as if you were comparing a history book with a book on Python using words of size 2. It's very likely that both books will contain the same words of size 2. Increasing the size of `k` will produce substantial differences. 

Change the function `get_kmer_2` so that given a sequence `X` and a k-mer size `k`, the function returns a boolean vector of all the words of size `k` in `X`. Cal the function `get_kmer`



The following code can be used to generate all DNA words of size `k`
```pyton
words_size_k = ["".join(prod) for prod in product(*([DNA]*k))]
```

Once done, use the code below to test your function

In [15]:
def get_kmer(X, k = 2):
    DNA = ["A", "C", "G", "T"]
    words_size_k = ["".join(prod) for prod in product(*([DNA]*k))]
    counts = pd.Series([0 for _ in words_size_k], index = words_size_k)
    words_in_X = set([X[i:i+k] for i in range(len(X)-k+1)])
    counts[list(words_in_X)] = 1
    return counts    

In [16]:
# TEST PROVIDED -- DO NOT REMOVE
X = "ACGTGATGATTG"

counts = get_kmer(X, k=1)
assert counts.tolist() == [1,1,1,1]


counts = get_kmer(X, k=3)
assert (counts[["ACG", "CGT", "GTG", "TGA", "GAT", "ATG", "ATT", "TTG"]] == 1).sum()  == 8
assert (counts.drop(["ACG", "CGT", "GTG", "TGA", "GAT", "ATG", "ATT", "TTG"]) == 0).sum()  == 56


### Q.9

* Compute the Jaccard similarity for the pairs `(X, Y)`, `(X, Z)`, `(Y, Z)` using `k= 5`


In [17]:
X = "ACGTACGTACGTACGT"
Y = "ACGTACAAACGTACGT"
Z = "TTTTACAAACGTTTTT"

A, B, C = get_kmer(X, 5), get_kmer(Y, 5), get_kmer(Z, 5)
print(f'(X, Y) -> {jaccard(A, B)}\n(X, Z) -> {jaccard(A, C)}\n(Y, Z) -> {jaccard(B, C)}')

(X, Y) -> 0.4
(X, Z) -> 0.0
(Y, Z) -> 0.29411764705882354


The appropriate word size varies with the length of the text, with larger words depicting similarity more accurately. However, large values of `k` are:
1. More computationally intensive to compute. With k = 12, there are $4^12 = 16,777,216$ words to compute for each sequence.

2. More likely to skew the distance between fairly similar sequences. For example `k=8`, the Jaccard index between `X` and `Y` is `0`, even though `X` and `Y` have only two mismatching characters. While this is an extreme case due to the fact that X and Y are short, the logic applies to longer sequences and larger values of `k`


![](https://www.dropbox.com/s/rhw5szbiohsqu7w/mismatches.png?dl=1)


In [18]:
A, B = get_kmer(X, 8), get_kmer(Y, 8)
print(f'(X, Y) -> {jaccard(A, B)}')

(X, Y) -> 0.08333333333333333


The code I used is provided as a reference below. The code took 7 hours to complete on a single machine and approximately 12 minutes on a larger server with 72 cores and 1TB of RAM. To parallelize the execution, I split the file into files that contain 1000 sequences each and used GNU Parallel to run each file on a single CPU core.

In [19]:
# CODE PROVIDED FOR ILLUTRATION -- DO NOT REMOVE
# RUNNING LOCALLY MAY TAKE A LONG TIME TO COMPLETE

# k = 8 
# DNA = ["A", "C", "G", "T"]
# words_size_k = ["".join(prod) for prod in product(*([DNA]*k))]

    
# def get_kmer_mod(X):
#     counts = pd.Series([0 for _ in words_size_k], index = words_size_k)
#     words_in_X = set([X[i:i+k] for i in range(0, len(X)-k+1)])
#     counts[list(words_in_X)] = 1
#     return counts   

# def replace_bad_nucs(seq):
#     for character in ['W', 'K', "Y", "M", 'H']:
#         seq = seq.replace(character, 'A') 
        
#     for character in ['R', 'S', 'D', "V", "B"]:
#         seq = seq.replace(character, 'C') 
        
#     seq = seq.replace("N", '') 
    
#     return seq

# all_counts = []
# all_names = []
# with tqdm(total=1000) as pbar:
#     for record in SeqIO.parse("myseq0.fa", 'fasta'):
#         all_names.append(record.id)
#         seq = replace_bad_nucs(str(record.seq))

#         counts = get_kmer_mod(seq)
#         all_counts.append(counts)
#         pbar.update(1)
    
# kmer_counts = pd.DataFrame(all_counts, index = all_names)
# kmer_counts.head()

### Hashing Sequences

We are interested in finding pairs of sequences that are very similar. However, comparing the sequences pairwise is not tractable since it would require carrying out $429282 * (429282 - 1) / 2 = 92_141_303_121$ comparisons.

Instead, we will use the hashing-based approach covered in class. Rather than hashing a sequence over all k-mers, we will only compute the hash for a subset of k-mers. we will repeat the operation n times to avoid that similar sequences are assigned to different bins due to a single, rare mismatch.

This, as discussed in class, is computationally more efficient compared to computing all pairwise sequences. 

### Q.10 

Write a function that takes a `pandas  Series` and a subset of columns and returns the hash computed on the subset of columns. Call this function`hash_on_subset`.

As an example, consider all words with a size of 2 as follows 

|	|AA	|AC	|AG	|AT	|CA | CC| CG| CT| GA| GC| GG| GT| TA| TC| TG| TT|
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
| A	|0	|1	|0	|0	|0	|0	|1	|0	| 0 |0	|0	| 1 |1  |0	|0	|0  |

```python
hash_on_subset(A, ["AC", "CG", "CT", "GT", "TA"]) 
```

is equivalent to:

```python
hash((1, 1, 0, 1, 1)) == 5085477689562523216
```


In [20]:
def hash_on_subset(pan, arr):
    res = []
    for i in arr:
        res.append(pan[i])
    return hash(tuple(res))

A = get_kmer_2(X)
hash_on_subset(A, ["AC", "CG", "CT", "GT", "TA"]) 

5085477689562523216


The method `sample` from the random module `m` words from a list

For example, running:
```python
random.sample( ["A", "C", "G", "T"], 2 )
```
returns
```
['A', 'C']
```
The returned subset may be different for you.

* The code below randomly selects `m=20` k-mers we will use to compare the genomes



In [21]:
random.sample( ["A", "C", "G", "T"], 2 )

['T', 'A']

In [22]:
k=8
DNA = ["A", "C", "G", "T"]
words_size_k = ["".join(prod) for prod in product(*([DNA]*k))]

m=20
subset_kmers = random.sample(words_size_k, m)

# subset_kmers

### Q.12


Apply the function `hash_subset` to all the rows of `all_kmers_df`. The data science (*vectorized*) way to do so is using the `apply` method available on a `pandas DataFrame` instead of using for loops. For example, given a DataFrame `df` such that:

```
df = pd.DataFrame([[1,2,3], [4,5,6]])

```
then 
```
df.apply(max, args=[] axis=1)
```
applies the `max()` function on each row (`axis = 1`). Here, `args` is empty since `max` does not take any additional arguments.

The example below shows how to use `apply` when the function requires additional arguments. In this example, we apply a function that sums all the values of a row and adds to the sum an offset (2 by default)



In [23]:
# EXAMPLE CODE PROVIDED -- DO NOT REMOVE
def add_val_to_sum(x, offset=2):
    return x.sum() + offset
    
df = pd.DataFrame([[1,2,3], [4,5,6]])

print("The sum of rows + an offset of 5 is:")
print(df.apply(add_val_to_sum, args=[5], axis=1))

print("The sum of rows + an offset of 10 is:")
print(df.apply(add_val_to_sum, args=[10], axis=1))
   

The sum of rows + an offset of 5 is:
0    11
1    20
dtype: int64
The sum of rows + an offset of 10 is:
0    16
1    25
dtype: int64


### Q.13


Use `apply()` to apply `hash_subset` and compute the hash values for all the rows of `all_kmers_df` over `subset_kmers`

* Create a dict by parsing the results to group sequences that yield the same hash under the same bins. Each key in the dict should be a key and each value is a list of sequences that have the same value.

For example, in the dictionary below, X and Y have the same hash value (123456) over a given subset of kmers, whereas Z has a different hash over the same subsets.

```
{"123456": [X,Y], "654321": [Z]}
```


In [24]:
# Takes a while
all_kmers_df = pd.read_csv('./data/all_kmers_10k.csv', low_memory=False)
result = all_kmers_df.apply(hash_on_subset, args=[subset_kmers], axis=1)
result_dict = {}
for i in range(len(result)):
    try:
        result_dict[result[i]].append(i)
    except KeyError:
        result_dict[result[i]] = [i]

result_dict

{-4057562519893178744: [0,
  2,
  3,
  4,
  5,
  6,
  8,
  9,
  10,
  11,
  12,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100,
  101,
  102,
  103,
  106,
  107,
  108,
  109,
  110,
  111,
  112,
  113,
  114,
  115,
  116,
  117,
  118,
  119,
  120,
  121,
  122,
  123,
  124,
  125,
  126,
  127,
  128,
  129,
  130,
  131,
  132,
  133,
  134,
  135,
  136,
  137,
  138,
  139,
  140,
  141,
  142,
  143,
  144,
  145,
  146,
  147,
  148,
  149,
  150,
  151,
  152,
  153,
  154,
  155,
  156,
  157,
  158,
  159,
  160,
  161,
  

### Q. 15

Here we use the presence and absence of words, i.e., a vector of booleans, to encode a sequence. The problem with this approach is that it considers the sequences to be identical, even if their word counts differ substantially. For example, given the sequence `X`, `Y` and `Z` as follows
```
X = ATAGATAGATAGATAGATT
Y = ATAGATAGATAGATAGATT
Z = ATAGATTTTTTTTTTTTTT
```
With k =2, all three sequences mach on their vector of word presence/absense.

In [25]:
X = "ATAGATAGATAGATAGATT"
Y = "ATAGATAGATAGATAGATT"
Z = "ATAGATTTTTTTTTTTTTT"

counts_dict = {"X": get_kmer(X, k=2), "Y": get_kmer(Y, k=2), "Z": get_kmer(Z, k=2)}
pretty_print_counts(counts_dict)

,AA,AC,AG,AT,CA,CC,CG,CT,GA,GC,GG,GT,TA,TC,TG,TT
X,0,0,1,1,0,0,0,0,1,0,0,0,1,0,0,1
Y,0,0,1,1,0,0,0,0,1,0,0,0,1,0,0,1
Z,0,0,1,1,0,0,0,0,1,0,0,0,1,0,0,1


* Comparing these sequences based on word counts, X is much more similar to Y than it is to Z

In [26]:
def get_kmer_counts(X, k):
    DNA = ["A", "C", "G", "T"]
    words_size_k = ["".join(prod) for prod in product(*([DNA]*k))]
    counts = pd.Series([0 for _ in words_size_k], index = words_size_k)
    counts_words_in_x = Counter([X[i:i+k] for i in range(0, len(X)-k+1)])
    counts.update(counts_words_in_x)
    return counts    


counts_dict = {"X": get_kmer_counts(X, k=2), "Y": get_kmer_counts(Y, k=2), "Z": get_kmer_counts(Z, k=2)}
pretty_print_counts(counts_dict)

,AA,AC,AG,AT,CA,CC,CG,CT,GA,GC,GG,GT,TA,TC,TG,TT
X,0,0,4,5,0,0,0,0,4,0,0,0,4,0,0,1
Y,0,0,4,5,0,0,0,0,4,0,0,0,4,0,0,1
Z,0,0,1,2,0,0,0,0,1,0,0,0,1,0,0,13


* Given an example (vectors) to justify why hashing is not ideal with counts.
  * Use any means you think are useful to illustrate your point (e.g.: figure, simulation (yes, please!))
 
* Describe how the random project approach discussed in class can help solve the issue discussed
  * Use code to illustrate how random projection works in the following example.
    * I.e., provide code to provide an example where `X` and `Y` are assigned to the same bin and `Y` is assigned to a different bin.
    * You can choose any vector values as needed 
 
```python
X = [1,2]
Y = [2,2]
Z = [5,1]
```

In [27]:
### Write your code here
vectors = {'X': (1,2), 'Y': (2, 2), 'Z': (5,1)}
print(f"Hash for X is {hash(vectors['X'])}, Hash for Y is {hash(vectors['Y'])}")
print('As seen above. X and Y are similar but have totally different hashes due to array of counts instead of booleans\n')

# Using similar code from the class notes
D = ((1+2+5)/3, (2+2+1)/3)

amp_D = (D[0]**2 + D[1]**2)**.5
print(f'D = {D}')
print(f"The magnitude of D is: {amp_D}\n")

for k, v in vectors.items():
    dot_D = vectors[k][0]*D[0] + vectors[k][1]*D[1]
    proj = dot_D / amp_D
    print(f'{k}:')
    print (f"\tThe magnitude of the projection  {proj}")
    print(f"\tThe ratio of the projection in terms of D is {proj/amp_D}")
    print(f"\tProjection occurs in bin {int(proj/amp_D)}\n")

Hash for X is -3550055125485641917, Hash for Y is 1901736143494378007
As seen above. X and Y are similar but have totally different hashes due to array of counts instead of booleans

D = (2.6666666666666665, 1.6666666666666667)
The magnitude of D is: 3.1446603773522015

X:
	The magnitude of the projection  1.9079961840114479
	The ratio of the projection in terms of D is 0.6067415730337078
	Projection occurs in bin 0

Y:
	The magnitude of the projection  2.7559944880165355
	The ratio of the projection in terms of D is 0.8764044943820223
	Projection occurs in bin 0

Z:
	The magnitude of the projection  4.769990460028619
	The ratio of the projection in terms of D is 1.5168539325842691
	Projection occurs in bin 1

